**Chapter 11 – Training Deep Neural Networks**

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/jdecorte/machinelearning/blob/main/110-training_deep_neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Problems when training (large) neural networks

- Sometimes you need to tackle a complex problem, such as detecting hundreds of types of objects in high-resolution images? 
- You may then need to train a much deeper DNN
  - with 10 layers or many more
  - each containing hundreds of neurons
  - linked by hundreds of thousands of connections
- When training you may the run into several problems: 
  - _vanishing gradients_ problem or the related _exploding gradients_ problem. This is when the gradients grow smaller and smaller, or larger and larger, when flowing backward through the DNN during training. Both of these problems make lower layers very hard to train.
  - **not have enough training** data for such a large network, or it might be too costly to label.
  - training may be extremely **slow**.
  - a model with millions of parameters would severely risk **overfitting** the training set, especially if there are not enough training instances or if they are too noisy.
- In this chapter we will go through each of these problems and present techniques to solve them.

# Vanishing/Exploding Gradients Problem
- As we discussed in the previous chapter, the backpropagation algorithm works by going from the output layer to the input layer, propagating the error gradient along the way. 
- Once the algorithm has computed the gradient of the cost function with regard to each parameter in the network, it uses these gradients to update each parameter with a Gradient Descent step.
- Unfortunately, gradients often get smaller and smaller as the algorithm progresses down to the lower layers: _vanishing gradients_
- In some cases, the gradients can grow bigger and bigger until layers get insanely large weight updates and the algorithm diverges: _exploding gradients_
- Recent (2000-2015) scientific research has revealed that these problems are caused mainly by the use of the sigmoid activation function, which _saturates_ when inputs become large (negative or positive), causing the gradients (which are derivatives) to become close to 0:
![](img/sigmoid_saturation_plot.png)
- Other activation functions we have seen before also suffer from satuaration for large positive and/or negative inputs: 
![](img/activation_functions_plot.png)

- To cope with this problem several **non saturating activation functions** have been proposed: 
  - Leaky ReLU: 
    ![](img/leaky_relu_plot.png)
  - ELU (Exponential Linear Unit)
    ![](img/elu_plot.png)
  - SELU (Scaled ELU)
    ![](img/selu_plot.png)



## Training with nonsaturating Activation Functions

### Leaky ReLU

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [2]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

- To use the leaky ReLU activation function, create a LeakyReLU layer and add it to your model just **after** the layer you want to apply it to. 
- We also illustrate the use of another weights initializer (`he_normal`). The mathematics behind this initializer are outside the scope of this course. 

In [3]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [5]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 2s 1ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.7955 - accuracy: 0.7362 - val_loss: 0.7130 - val_accuracy: 0.7658
Epoch 3/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900
Epoch 4/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8198
Epoch 6/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5553 - accuracy: 0.8157 - val_loss: 0.5350 - val_accuracy: 0.8238
Epoch 7/10
1719/1719 [==============================] - 2s 1ms/step - loss: 0.5339 - accuracy: 0.8224 - val_loss: 0.5157 - val_accuracy:

### ELU

Implementing ELU in TensorFlow is trivial, just specify the activation function when building each layer:

In [20]:
keras.layers.Dense(10, activation="elu")

### SELU

It can be shown that: 
- If you build a neural network composed exclusively of a stack of dense layers, and if all hidden layers use the SELU activation function, then the network will self-normalize: the output of each layer will tend to preserve a mean of 0 and standard deviation of 1 during training, which solves the vanishing/exploding gradients problem.
- As a result, the SELU activation function often significantly outperforms other activation functions for such neural nets (especially deep ones).
- There are, however, a few conditions for self-normalization to happen:
  - The input features must be standardized (mean 0 and standard deviation 1).
  - Every hidden layer’s weights must be initialized with a special initialisation, called _LeCun normal initialization_. In Keras, this means setting kernel_initializer="lecun_normal".
  - The network’s architecture must be sequential (so no loops al in recurrent networks)

Using SELU is easy:

In [6]:
keras.layers.Dense(10, activation="selu",
                   kernel_initializer="lecun_normal")

Let's create a neural net for Fashion MNIST with 100 hidden layers, using the SELU activation function:

In [7]:
np.random.seed(42)
tf.random.set_seed(42)

In [10]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="selu",
                             kernel_initializer="lecun_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="selu",
                                 kernel_initializer="lecun_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [11]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

Now let's train it. Do not forget to scale the inputs to mean 0 and standard deviation 1:

In [12]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [13]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 15s 8ms/step - loss: 1.0808 - accuracy: 0.5875 - val_loss: 0.7535 - val_accuracy: 0.7268
Epoch 2/5
1719/1719 [==============================] - 13s 8ms/step - loss: 0.6618 - accuracy: 0.7571 - val_loss: 0.5921 - val_accuracy: 0.7878
Epoch 3/5
1719/1719 [==============================] - 14s 8ms/step - loss: 0.5599 - accuracy: 0.7945 - val_loss: 0.6036 - val_accuracy: 0.7906
Epoch 4/5
1719/1719 [==============================] - 15s 9ms/step - loss: 0.5413 - accuracy: 0.8059 - val_loss: 0.4919 - val_accuracy: 0.8286
Epoch 5/5
1719/1719 [==============================] - 15s 9ms/step - loss: 0.4871 - accuracy: 0.8265 - val_loss: 0.4585 - val_accuracy: 0.8440


Now look at what happens if we try to use the ReLU activation function instead:

In [14]:
np.random.seed(42)
tf.random.set_seed(42)

In [15]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))
for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))
model.add(keras.layers.Dense(10, activation="softmax"))

In [17]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [18]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 12s 6ms/step - loss: 1.7702 - accuracy: 0.2780 - val_loss: 1.4824 - val_accuracy: 0.3470
Epoch 2/5
1719/1719 [==============================] - 11s 6ms/step - loss: 1.2034 - accuracy: 0.4837 - val_loss: 1.0297 - val_accuracy: 0.5528
Epoch 3/5
1719/1719 [==============================] - 11s 6ms/step - loss: 1.0226 - accuracy: 0.5759 - val_loss: 0.8916 - val_accuracy: 0.6216
Epoch 4/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.8909 - accuracy: 0.6501 - val_loss: 0.7518 - val_accuracy: 0.7158
Epoch 5/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.7646 - accuracy: 0.7095 - val_loss: 0.7669 - val_accuracy: 0.7130


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

- To avoid the vanishing/exploding gradient problem we can also explicitly normalize the output of a layer. 
- This technique is called _batch normalization_.
- It consists of adding an operation in the model just before or after the activation function of each hidden layer. 
- This operation simply zerocenters and normalizes each input.

In [19]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [20]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_408 (Dense)           (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_409 (Dense)           (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [21]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [22]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 0.8750 - accuracy: 0.7123 - val_loss: 0.5525 - val_accuracy: 0.8228
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5753 - accuracy: 0.8030 - val_loss: 0.4724 - val_accuracy: 0.8476
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5189 - accuracy: 0.8206 - val_loss: 0.4375 - val_accuracy: 0.8546
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4827 - accuracy: 0.8322 - val_loss: 0.4153 - val_accuracy: 0.8598
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4565 - accuracy: 0.8407 - val_loss: 0.3997 - val_accuracy: 0.8642
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4397 - accuracy: 0.8473 - val_loss: 0.3868 - val_accuracy: 0.8694
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.4242 - accuracy: 0.8515 - val_loss: 0.3763 - val_accuracy:

## Reusing Pretrained Layers (Transfer Learning)
- it is generally not a good idea to train a very large DNN from scratch
- instead, you should always try to find an existing neural network that accomplishes a similar task to the one you are trying to tackle
- then reuse the lower layers of this network
- this technique is called _transfer learning_. 
  - it speeds up training considerably
  - it requires significantly less training data

![](img/pretrained_layers.PNG)

- The output layer of the original model should usually be replaced because it is most likely not useful at all for the new task, and it may not even have the right number of outputs for the new task.
- Similarly, the upper hidden layers of the original model are less likely to be as useful as the lower layers, since the high-level features that are most useful for the new task may differ significantly from the ones that were most useful for the original task. 
- Reuse existing layers with Keras is very simple: 

In [ ]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])  # reuse all layers except the output layer
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))   

Note that `model_B_on_A` and `model_A` actually share layers now, so when we train one, it will update both models. If we want to avoid that, we need to build `model_B_on_A` on top of a *clone* of `model_A`:

In [61]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# Faster Optimizers

Training a very large deep neural network can be painfully slow. So far we have seen four ways to speed up training (and reach a better solution):
- applying a good initialization strategy for the connection weights, 
- using a good activation function
- using Batch Normalization
- reusing parts of a pretrained network 
  
Another huge speed boost comes from using a faster optimizer (for finding the minimum of the cost function) than the regular Gradient Descent optimizer. 

The most popular algorithms are: 
- Momentum optimization
- Nesterov Accelerated Gradient
- AdaGrad
- RMSProp
- Adam and Nadam optimization

We don't go into the mathematical details of these algorithms but keep in mind they can be useful when finetuning your network. 

## Momentum optimization

In [67]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

## Nesterov Accelerated Gradient

In [68]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

## AdaGrad

In [69]:
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)

## RMSProp

In [70]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

## Adam Optimization

In [71]:
optimizer = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

## Adamax Optimization

In [72]:
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

## Nadam Optimization

In [73]:
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# Avoiding Overfitting Through Regularization

## Dropout
_Dropout_ is one of the most popular regularization techniques for deep neural networks.

- it is a simple algorithm: at every training step, every neuron (including the input neurons, but always excluding the output neurons) has a probability $p$ of being temporarily “dropped out” 
- it will the be entirely ignored during this training step, but it may be active during the next step
- the hyperparameter p is called the dropout rate, and it is typically set between 10% and 50%
- we can understand the power of dropout by realizing that a unique neural network is generated at each training step.
  
  ![](img/dropout.png)

- To implement dropout using Keras, you can use the keras.layers.Dropout layer. 
- During training, it randomly drops some inputs (setting them to 0).
- If you observe that the model is overfitting, you can increase the dropout rate (and vice versa).

In [23]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 5s 2ms/step - loss: 0.5755 - accuracy: 0.8031 - val_loss: 0.3721 - val_accuracy: 0.8630
Epoch 2/2
1719/1719 [==============================] - 4s 2ms/step - loss: 0.4217 - accuracy: 0.8449 - val_loss: 0.3483 - val_accuracy: 0.8704
